In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Create network
model = Network('model')

# Block 1: nodes
node1 = Delay(model, 'Delay')
node2 = Queue(model, 'Queue1', SchedStrategy.PS)
node3 = Queue(model, 'Queue2', SchedStrategy.FCFS)  # FCFS instead of PS
node3.set_number_of_servers(2)

In [3]:
# Block 2: classes
N = [2, 0]  # Class1: 2 jobs, Class2: 0 jobs
jobclass1 = ClosedClass(model, 'Class1', N[0], node1, 0)
jobclass2 = ClosedClass(model, 'Class2', N[1], node1, 0)

In [4]:
# Set service times
node1.set_service(jobclass1, Exp.fitMean(1.0))
node1.set_service(jobclass2, Exp.fitMean(1.0))

node2.set_service(jobclass1, Exp.fitMean(1.0/3.0))
node2.set_service(jobclass2, Exp.fitMean(1.0/4.0))

# FCFS queue with same service time for both classes
node3.set_service(jobclass1, Exp.fitMean(1.0/3.0))
node3.set_service(jobclass2, Exp.fitMean(1.0/3.0))

In [5]:
# Block 3: routing with class switching
P = model.init_routing_matrix()

# Class1 routing: Delay -> Queue1 (stay Class1), Queue2 -> Delay (switch to Class2)
P.set(jobclass1, jobclass1, node1, node2, 1.0)  # Delay -> Queue1
P.set(jobclass1, jobclass2, node2, node3, 1.0)  # Queue1 -> Queue2 (switch to Class2)

# Class2 routing: Queue2 -> Delay (switch to Class1)
P.set(jobclass2, jobclass1, node1, node2, 1.0)  # Delay -> Queue1 (switch to Class1)
P.set(jobclass2, jobclass2, node2, node3, 1.0)  # Queue1 -> Queue2
P.set(jobclass2, jobclass1, node3, node1, 1.0)  # Queue2 -> Delay (switch to Class1)

model.link(P)

In [6]:
# Set specific state for probability calculation
n = np.array([[0,0],   # Delay: no jobs
              [1,0],   # Queue1: 1 job of Class1
              [0,1]])  # Queue2: 1 job of Class2

nodes = [node1, node2, node3]
for i in range(len(nodes)):
    nodes[i].setState(n[i])

state = model.getState()

In [7]:
# Solver options
options = Solver.defaultOptions
options.verbose = 1
options.samples = 20000
options.seed = 23001  # Different seed from example 5

In [9]:
# Method 1: getProbAggr - Marginal probabilities for aggregated state
solver_ctmc = SolverCTMC(model, options)
Pr = solver_ctmc.getProbAggr(node3)

java.lang.IllegalArgumentException: java.lang.IllegalArgumentException: Outside of matrix bounds

In [ ]:

solver_nc = SolverNC(model, options)
Pr = solver_nc.getProbAggr(node3)

In [ ]:

solver_ssa = SolverSSA(model, options)
Pr = solver_ssa.getProbAggr(node3)

In [ ]:

solver_jmt = SolverJMT(model, options)
Pr = solver_jmt.getProbAggr(node3)

In [ ]:
# Method 2: getProb - Marginal probabilities for detailed state
solver_ctmc = SolverCTMC(model, options)
Pr = solver_ctmc.getProb(node3)

In [ ]:

solver_ssa = SolverSSA(model, options)
Pr = solver_ssa.getProb(node3)

In [ ]:
# Method 3: getProbSysAggr - Joint system probabilities for aggregated state
solver_ctmc = SolverCTMC(model, options)
Pr = solver_ctmc.getProbSysAggr()

In [ ]:

solver_nc = SolverNC(model, options)
Pr = solver_nc.getProbSysAggr()

In [ ]:

solver_ssa = SolverSSA(model, options)
Pr = solver_ssa.getProbSysAggr()

In [ ]:

solver_jmt = SolverJMT(model, options)
Pr = solver_jmt.getProbSysAggr()

In [ ]:
# Method 4: getProbSys - Joint system probabilities for detailed state
solver_ctmc = SolverCTMC(model, options)
Pr = solver_ctmc.getProbSys()

In [ ]:

solver_ssa = SolverSSA(model, options)
Pr = solver_ssa.getProbSys()